# Set up environments

In [4]:
!pip install -q beautifulsoup4==4.12.2 PyMuPDFb==1.24.10 tqdm==4.66.5

In [5]:
import os

if not ".git" in os.listdir():
    !git clone https://github.com/batprem/pyo3-experiment
    os.chdir("pyo3-experiment")

# Download 56-1 file

In [47]:
import requests
from bs4 import BeautifulSoup
import os
import zipfile
from tqdm import tqdm


ROOT = "downloaded-56-1"
SYMBOL = "PTT"  # Change this symbol
# SYMBOL = "KBANK"  # Change this symbol
FILE_PATH = f"{ROOT}/{SYMBOL}"
PATH_TO_TARGET = f"{FILE_PATH}/{SYMBOL}.zip"
session = requests.Session()



os.makedirs(FILE_PATH, exist_ok=True)
print("Searching the balance sheet file")

response = session.get(
    f"https://market.sec.or.th/public/idisc/th/Viewmore/fs-r561?searchSymbol={SYMBOL}",
)
soup = BeautifulSoup(response.text)
url = soup.find_all("tr")[1].find_all("td")[-1].find("a")["href"]

print(f"Found at {url}")
# Sizes in bytes.
total_size = int(response.headers.get("content-length", 0))
block_size = 1024
response = session.get(url, stream=True)

print("Downloading...")
with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
    with open(PATH_TO_TARGET, "wb") as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)

    
# with open(PATH_TO_TARGET, "wb") as f:
#     f.write(requests.get(url).content)

with zipfile.ZipFile(PATH_TO_TARGET, 'r') as zip_ref:
    zip_ref.extractall(FILE_PATH)
print("Done")

Searching the balance sheet file
Found at https://market.sec.or.th/public/idisc/Download?FILEID=dat/f56/0646ONE140320241323370901T.zip
Downloading...


44.0MB [00:04, 10.6MB/s]                                                          


Done


# Extract table of contents

In [48]:
import fitz
import pandas as pd

def to_html(filepath: str):
    doc = fitz.open(filepath)
    for i, page in enumerate(doc):
        text = page.get_text("html")
        with open(f"pymupdf-page-{i}.html", "w") as fp:
            fp.write(text)
    doc.close()
    
doc = fitz.open(f'{FILE_PATH}/STRUCTURE{SYMBOL}T.PDF')
toc = pd.DataFrame(doc.get_toc())
toc.columns = ["hierachy", "topic", "page"]
toc = toc[toc["topic"] != "Bookmark"].reset_index(drop=True)
toc

,hierachy,topic,page
0,1,Cover,1
1,1,TableContent,2
2,1,StructureAndOperation,4
3,1,RiskManagement,9
4,1,ESG,13
5,1,CorporateGovernancePolicy,18
6,1,CorporateGovernanceStructure,20
7,1,ResultOfCorporateGovernance,32


In [49]:
first_page_index = toc.index[toc["topic"] == "StructureAndOperation"][0]
end_page_index = first_page_index + 1
first_page, end_page = toc.loc[[first_page_index, end_page_index]]["page"].to_list()

In [50]:
first_page, end_page

(4, 9)

In [56]:
from IPython.display import display

structure_type = "Not defined"
for page in range(first_page, end_page):
    for table in doc[page].find_tables():
        df = table.to_pandas()
        if "Col1" in df.columns:
            columns_list = df.columns.to_list()
            columns_list[1:] = df.iloc[0].to_list()[1:]
            structure_type = columns_list[0]
            columns_list[0] = "detail"
            df.columns = columns_list
            df = df.iloc[1:]
            df["structure_type"] = structure_type
        if "Col0" in df.columns:
            df.columns = columns_list
            df["structure_type"] = structure_type
        display(df)

,detail,2564,2565,2566,structure_type
1,รายได้จากการดำเนินงานรวม (พันบาท),"2,258,818,487.07","3,367,202,701.45","3,144,550,828.53",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
2,สำรวจและผลิตปิโตรเลียม (พันบาท),"63,564,280.16","83,469,331.58","76,240,254.24",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
3,ก๊าซธรรมชาติ (พันบาท),"317,521,958.80","509,391,306.43","471,875,240.24",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
4,น้ำมัน (พันบาท),"503,914,383.50","776,455,993.90","736,805,268.46",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
5,ปิโตรเคมีและการกลั่น (พันบาท),"684,646,836.24","891,720,920.72","785,754,626.96",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
6,การค้าระหว่างประเทศ (พันบาท),"614,373,908.07","957,645,560.70","979,239,928.97",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
7,ธุรกิจใหม่และโครงสร้างพื้นฐาน (พันบาท),"58,390,414.48","116,948,081.02","92,576,391.31",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
8,อื่นๆ (พันบาท),"16,406,705.82","31,571,507.10","2,059,118.35",โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ


,detail,2564,2565,2566,structure_type
0,รายได้จากการดำเนินงานรวม (%),100.00,100.00,100.00,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
1,สำรวจและผลิตปิโตรเลียม (%),2.81,2.48,2.42,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
2,ก๊าซธรรมชาติ (%),14.06,15.13,15.01,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
3,น้ำมัน (%),22.31,23.06,23.43,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
4,ปิโตรเคมีและการกลั่น (%),30.31,26.48,24.99,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
5,การค้าระหว่างประเทศ (%),27.20,28.44,31.14,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
6,ธุรกิจใหม่และโครงสร้างพื้นฐาน (%),2.58,3.47,2.94,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ
7,อื่นๆ (%),0.73,0.94,0.07,โครงสร้างรายได้ของสายผลิตภัณฑ์หรือกลุ่มธุรกิจ


,detail,2564,2565,2566,structure_type
1,รายได้จากการดำเนินงานรวม (พันบาท),"2,258,818,487.07","3,367,202,701.45","3,144,550,828.53",โครงสร้างรายได้จากในประเทศและจากต่างประเทศ
2,รายได้จากในประเทศ (พันบาท),"1,329,573,212.58","1,933,531,559.66","1,744,025,735.77",โครงสร้างรายได้จากในประเทศและจากต่างประเทศ
3,รายได้จากต่างประเทศ (พันบาท),"929,245,274.49","1,433,671,141.79","1,400,525,092.76",โครงสร้างรายได้จากในประเทศและจากต่างประเทศ


,detail,2564,2565,2566,structure_type
0,รายได้จากการดำเนินงานรวม (%),100.00,100.00,100.00,โครงสร้างรายได้จากในประเทศและจากต่างประเทศ
1,รายได้จากในประเทศ (%),58.86,57.42,55.46,โครงสร้างรายได้จากในประเทศและจากต่างประเทศ
2,รายได้จากต่างประเทศ (%),41.14,42.58,44.54,โครงสร้างรายได้จากในประเทศและจากต่างประเทศ


,detail,2564,2565,2566,structure_type
1,รายได้อื่นรวม (พันบาท),"28,940,025.23","24,420,164.77","40,705,246.65",รายได้อื่น ตามที่ระบุในงบการเงิน
2,รายได้อื่นจากการดำเนินงาน (พันบาท),0.00,0.00,0.00,รายได้อื่น ตามที่ระบุในงบการเงิน
3,รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท),0.00,0.00,0.00,รายได้อื่น ตามที่ระบุในงบการเงิน


,detail,2564,2565,2566,structure_type
0,ส่วนแบ่งกำไร (ขาดทุน) (พันบาท),"9,010,122.59","5,848,984.59","3,012,624.96",รายได้อื่น ตามที่ระบุในงบการเงิน


,รายการ (หน่วย : พันบาท),2564,2565,2566
0,ค่าใช้จ่ายการทำวิจัยและพัฒนา (R&D) ในระยะเวลา ...,N/A,N/A,N/A


In [42]:
table.to_pandas().columns

Index(['รายการ (หน่วย : พันบาท)', '2564', '2565', '2566'], dtype='object')

# Extract tables

In [15]:
table = doc[5].find_tables()[0]
# print(doc.get_page_text(5))
table.to_pandas()

,รายได้อื่น ตามที่ระบุในงบการเงิน,Col1,Col2,Col3
0,,2564,2565,2566
1,รายได้อื่นรวม (พันบาท),0.00,0.00,0.00
2,รายได้อื่นจากการดำเนินงาน (พันบาท),0.00,0.00,0.00
3,รายได้อื่นนอกเหนือจากการดำเนินงาน (พันบาท),0.00,0.00,0.00


In [39]:
table = doc[13].find_tables()[1]
# print(doc.get_page_text(5))
table.to_pandas()

,• การฝึกอบรมและพัฒนาพนักงาน,Col1,Col2,Col3
0,,2564,2565,2566
1,จำนวนชั่วโมงอบรมเฉลี่ยของพนักงาน (ชั่วโมง/คน/ปี),37.40,40.40,34.50


In [47]:
table = doc[17].find_tables()[1]
# print(doc.get_page_text(5))
table.to_pandas()

,รายชื่อกรรมการชุดปัจจุบัน,Col1,Col2,Col3
0,ข้อมูลทั่วไป,ตำแหน่งของกรรมการ,วันเริ่มดำรงตำแหน่ง,ประสบการณ์และความชำนาญ
1,1. นาง กอบกาญจน์ วัฒนวรางกูร\nเพศ: หญิง\nอายุ:...,"ประธานกรรมการ\n(กรรมการที่ไม่เป็นผู้บริหาร,\nก...",4 เม.ย. 2561,"ความยั่งยืน, การจัดการกลยุทธ์, การ\nจัดการความ..."
2,2. นางสาว สุจิตพรรณ ล่ำซำ\nเพศ: หญิง\nอายุ: 64...,รองประธานกรรมการ\n(กรรมการที่ไม่เป็นผู้บริหาร)...,30 เม.ย. 2541,"การเงิน, ความยั่งยืน, ธนาคาร, การ\nจัดการกลยุท..."
